In [1]:
import os, sys, re
import json
from watson_developer_cloud import AlchemyLanguageV1
import pandas as pd
import networkx as nx

In [2]:
base_dir = "/Users/singhv/Documents/Curation/thesaurus"

In [3]:
API_KEY = 'e0759019e81f7776b95cbebf8b2f2ffe0244a3f9'
MODEL_ID = '12fc2544-de5e-44bb-937f-a775e61e7c63'
alchemy_language = AlchemyLanguageV1(api_key=API_KEY)

In [3]:
temp = "code <tab> concept name <tab> parents <tab> synonyms <tab> definition <tab> display name <tab> concept status <tab> semantic type <EOL>"
headers = [x.strip() for x in temp[11:-5].split("<tab>")]
print headers

['concept name', 'parents', 'synonyms', 'definition', 'display name', 'concept status', 'semantic type']


In [4]:
thesaurus = pd.read_csv(os.path.join(base_dir, "Thesaurus.txt"), sep="\t", index_col=0, names=headers)

In [5]:
G = nx.DiGraph()

In [6]:
for i, (code, row) in enumerate(thesaurus.iterrows()):
    if i % 1000 == 0:
        print "Processed %d rows" % i
    parent_codes = row.get('parents')
    if code not in G:
        G.add_node(code)
    G.node[code]['name'] = row['concept name']
    G.node[code]['synonyms'] = row.get('synonyms', '').split('|')
    if parent_codes != 'root_node':
        parent_codes = parent_codes.split("|")
        for parent_code in parent_codes:
            if (parent_code not in G):
                G.add_node(parent_code)
            G.add_edge(parent_code, code)

Processed 0 rows
Processed 1000 rows
Processed 2000 rows
Processed 3000 rows
Processed 4000 rows
Processed 5000 rows
Processed 6000 rows
Processed 7000 rows
Processed 8000 rows
Processed 9000 rows
Processed 10000 rows
Processed 11000 rows
Processed 12000 rows
Processed 13000 rows
Processed 14000 rows
Processed 15000 rows
Processed 16000 rows
Processed 17000 rows
Processed 18000 rows
Processed 19000 rows
Processed 20000 rows
Processed 21000 rows
Processed 22000 rows
Processed 23000 rows
Processed 24000 rows
Processed 25000 rows
Processed 26000 rows
Processed 27000 rows
Processed 28000 rows
Processed 29000 rows
Processed 30000 rows
Processed 31000 rows
Processed 32000 rows
Processed 33000 rows
Processed 34000 rows
Processed 35000 rows
Processed 36000 rows
Processed 37000 rows
Processed 38000 rows
Processed 39000 rows
Processed 40000 rows
Processed 41000 rows
Processed 42000 rows
Processed 43000 rows
Processed 44000 rows
Processed 45000 rows
Processed 46000 rows
Processed 47000 rows
Proce

In [19]:
def make_bigrams(lst):
    return zip( lst[:-1], lst[1:] ) if len(lst) > 1 else ([(lst[0], )] if len(lst) == 1 else [])

In [24]:
#print nx.dfs_successors(G, 'C4872')
successors = nx.dfs_successors(G, 'C4872')
successors_codes = successors.keys() + sum(successors.values(), [])
print successors_codes

['C3771', 'C9134', 'C3995', 'C5141', 'C7768', 'C2924', 'C88375', 'C5166', 'C88376', 'C5214', 'C5164', 'C27829', 'C5163', 'C5160', 'C90513', 'C4001', 'C2918', 'C4194', 'C4190', 'C3641', 'C7950', 'C36085', 'C9245', 'C53553', 'C28432', 'C8287', 'C5175', 'C5177', 'C7769', 'C4017', 'C40347', 'C47857', 'C7688', 'C3862', 'C27832', 'C91230', 'C47858', 'C4872', 'C40361', 'C40364', 'C76328', 'C4190', 'C6870', 'C28311', 'C36301', 'C36300', 'C36303', 'C36302', 'C5140', 'C5454', 'C5455', 'C66933', 'C5137', 'C5138', 'C7949', 'C9456', 'C9457', 'C85835', 'C85836', 'C5167', 'C5168', 'C7782', 'C88377', 'C7770', 'C3771', 'C47857', 'C40354', 'C5160', 'C4017', 'C40360', 'C5163', 'C40364', 'C27831', 'C27830', 'C27832', 'C6987', 'C7690', 'C7767', 'C7769', 'C7696', 'C9246', 'C8607', 'C36107', 'C40351', 'C40350', 'C9132', 'C7362', 'C7645', 'C5139', 'C4195', 'C4018', 'C2924', 'C9136', 'C54691', 'C97049', 'C97051', 'C97052', 'C97053', 'C36084', 'C7439', 'C9135', 'C4189', 'C9131', 'C97965', 'C5142', 'C7950', 'C91

In [25]:
bigrams = []
successors = nx.dfs_successors(G, 'C4872')
for node_code in successors.keys() + sum(successors.values(), []):
    print "Processing %s" % (node_code)
    node = G.node[node_code]
    synonyms = node["synonyms"]
    for name in synonyms:
        json_result = alchemy_language.entities(model=MODEL_ID, text=name, language='english')
        if json_result and json_result.get("status") == "OK":
            entities = [ x["type"] for x in json_result["entities"] ]
            bigrams.extend(make_bigrams(entities))

Processing C3771
Processing C9134
Processing C3995
Processing C5141
Processing C7768
Processing C2924
Processing C88375
Processing C5166
Processing C88376
Processing C5214
Processing C5164
Processing C27829
Processing C5163
Processing C5160
Processing C90513
Processing C4001
Processing C2918
Processing C4194
Processing C4190
Processing C3641
Processing C7950
Processing C36085
Processing C9245
Processing C53553
Processing C28432
Processing C8287
Processing C5175
Processing C5177
Processing C7769
Processing C4017
Processing C40347
Processing C47857
Processing C7688
Processing C3862
Processing C27832
Processing C91230
Processing C47858
Processing C4872
Processing C40361
Processing C40364
Processing C76328
Processing C4190
Processing C6870
Processing C28311
Processing C36301
Processing C36300
Processing C36303
Processing C36302
Processing C5140
Processing C5454
Processing C5455
Processing C66933
Processing C5137
Processing C5138
Processing C7949
Processing C9456
Processing C9457
Processing

In [29]:
print len(bigrams)

1490


In [28]:
from collections import Counter
counter = Counter(bigrams)
df = pd.DataFrame(columns=['Bigram', 'Count'])
#print counter
count = 0
for (bigram, count) in counter.most_common(30):
    print "%s (%d)" % (" AND ".join(bigram), count)
    df.loc[count] = (" AND ".join(bigram), count)
    count += 1
    
df.to_excel("/Users/singhv/Desktop/BigramEntityFrequency.xlsx")

Anatomy_Specific_Entity AND Cancer_Entity (459)
Cancer_Entity AND Anatomy_Specific_Entity (248)
Tissue_Entity AND Cancer_Entity (234)
Locomotive_Modifier_Entity AND Tissue_Entity (87)
Stage_Entity AND Anatomy_Specific_Entity (72)
Tissue_Entity AND Anatomy_Specific_Entity (61)
Anatomy_Specific_Entity (60)
Locomotive_Modifier_Entity AND Anatomy_Specific_Entity (41)
Cancer_Entity (25)
Cancer_Entity AND Stage_Entity (22)
Disease_Entity AND Anatomy_Specific_Entity (22)
Disease_Modifier_Entity AND Anatomy_Specific_Entity (15)
Tissue_Entity AND Tissue_Entity (15)
Locomotive_Modifier_Entity AND Cancer_Entity (14)
Anatomy_Specific_Entity AND Tissue_Entity (14)
Cancer_Entity AND Disease_Modifier_Entity (12)
Finding_Lesion_Entity AND Tissue_Entity (12)
Cancer_Entity AND Tissue_Entity (8)
Disease_Modifier_Entity AND Tissue_Entity (7)
Anatomy_Specific_Entity AND Locomotive_Modifier_Entity (7)
Anatomy_Specific_Entity AND Anatomy_Specific_Entity (6)
Pos_Neg_Entity AND Anatomy_Specific_Entity (6)
Anat

In [7]:
json_result = alchemy_language.entities(model=MODEL_ID, text='Breast Columnar Cell Mucinous Carcinoma', language='english')

In [8]:
print json_result

{u'status': u'OK', u'usage': u'By accessing AlchemyAPI or using information generated by AlchemyAPI, you are agreeing to be bound by the AlchemyAPI Terms of Use: http://www.alchemyapi.com/company/terms.html', u'model': u'12fc2544-de5e-44bb-937f-a775e61e7c63', u'language': u'english', u'entities': [{u'count': u'1', u'text': u'Breast', u'type': u'Anatomy_Specific_Entity'}, {u'count': u'1', u'text': u'Mucinous', u'type': u'Tissue_Entity'}, {u'count': u'1', u'text': u'Carcinoma', u'type': u'Cancer_Entity'}]}


In [10]:
print json_result["entities"]

[{u'count': u'1', u'text': u'Carcinoma', u'type': u'Cancer_Entity'}, {u'count': u'1', u'text': u'Invasive', u'type': u'Locomotive_Modifier_Entity'}, {u'count': u'1', u'text': u'Lobular', u'type': u'Tissue_Entity'}, {u'count': u'1', u'text': u'mammary', u'type': u'Anatomy_Specific_Entity'}]
